# ✔ 9. [**Persistence de modèle**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/9_model_persistence.ipynb)<br/>([*Model persistence*](https://scikit-learn.org/stable/model_persistence.html))

Après l'entraînement d'un modèle scikit-learn, il est souhaitable d'avoir un moyen de conserver ce modèle pour une utilisation future sans avoir à recycler. Les sections suivantes vous donnent quelques conseils sur la manière de conserver un modèle scikit-learn.

✔ 9.1. [**Sérialisation spécifique Python**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/9_model_persistence.ipynb#python-specific-serialization)
([*Python specific serialization*](https://scikit-learn.org/stable/model_persistence.html#python-specific-serialization))

* ✔ 9.1.1. [**Limites de sécurité et de maintenabilité**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/9_model_persistence.ipynb#security-maintainability-limitations)
([*Security & maintainability limitations*](https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations))

* ✔ 9.1.2 [**Un format plus sécurisé : `skops`**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/9_model_persistence.ipynb#"a-more-secure-format-skops)([*A more secure format: `skops`*](https://scikit-learn.org/stable/model_persistence.html#"a-more-secure-format-skops))

✔ 9.2. [**Formats interopérables**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/9_model_persistence.ipynb#interoperable-formats)
([*Interoperable formats*](https://scikit-learn.org/stable/model_persistence.html#interoperable-formats))

<a id='python-specific-serialization'></a>

# 9.1. Sérialisation spécifique Python

Il est possible de sauvegarder un modèle dans scikit-learn en utilisant le modèle de persistance intégré de Python, à savoir [pickle](https://docs.python.org/3/library/pickle.html) :

In [1]:
from sklearn import svm
from sklearn import datasets
clf = svm.SVC()
X, y= datasets.load_iris(return_X_y=True)
clf.fit(X, y)
# SVC()

import pickle
s = pickle.dumps(clf)
clf2 = pickle.loads(s)
clf2.predict(X[0:1])
# array([0])
y[0]
# 0

0

Dans le cas spécifique de scikit-learn, il peut être préférable d'utiliser le remplacement de `pickle` (`dump` & `load`) par `joblib`, qui est plus efficace sur les objets qui comportent de grands tableaux `numpy` en interne, comme c'est souvent le cas pour les estimateurs ajustés de scikit-learn, mais qui ne peut *pickler* que sur le disque et non sur une chaîne :

In [3]:
from joblib import dump, load
dump(clf, 'filename.joblib')

['filename.joblib']

Plus tard, vous pouvez recharger le modèle picklé (éventuellement dans un autre processus Python) avec :

In [4]:
clf = load('filename.joblib')

**NB** - les fonctions `dump` et `load` acceptent également les objets de type fichier au lieu des noms de fichiers. Plus d'informations sur la persistance des données avec [Joblib](https://joblib.readthedocs.io/en/latest/persistence.html) sont disponibles ici.

<a id='security-maintainability-limitations'></a>

## 9.1.1. Limites de sécurité et de maintenabilité

pickle (et joblib par extension), a quelques problèmes concernant la maintenabilité et la sécurité. À cause de cela,
* Ne décryptez jamais les données non fiables, car cela pourrait entraîner l'exécution d'un code malveillant lors du chargement.
* Bien que les modèles enregistrés à l'aide d'une version de scikit-learn puissent se charger dans d'autres versions, cela n'est absolument pas pris en charge et déconseillé. Il convient également de garder à l'esprit que les opérations effectuées sur ces données pourraient donner des résultats différents et inattendus.

Afin de reconstruire un modèle similaire avec les futures versions de scikit-learn, des métadonnées supplémentaires doivent être enregistrées avec le modèle picklé :
* Les données d'entraînement, par ex. une référence à un instantané immuable
* Le code source Python utilisé pour générer le modèle
* Les versions de scikit-learn et ses dépendances
* Le score de validation croisée obtenu sur les données d'apprentissage

Cela devrait permettre de vérifier que le score de validation croisée est dans la même fourchette qu'auparavant.

À quelques exceptions près, les modèles picklés devraient être portables sur toutes les architectures en supposant que les mêmes versions de dépendances et de Python sont utilisées. Si vous rencontrez un estimateur qui n'est pas portable, veuillez ouvrir un problème sur GitHub. Les modèles picklés sont souvent déployés en production à l'aide de conteneurs, comme Docker, afin de figer l'environnement et les dépendances.

Si vous souhaitez en savoir plus sur ces problèmes et explorer d'autres méthodes de sérialisation possibles, veuillez vous reporter à cette [conférence d'Alex Gaynor](https://pyvideo.org/pycon-us-2014/pickles-are-for-delis-not-software.html).

<a id='a-more-secure-format-skops'></a>

## 9.1.2. Un format plus sécurisé : `skops`

[**`skops`**](https://skops.readthedocs.io/en/stable/) fournit un format plus sécurisé via le module [**`skops.io`**](https://skops.readthedocs.io/en/stable/modules/classes.html#module-skops.io). Il évite d'utiliser `pickle` et ne charge que des fichiers ayant des types et des références à des fonctions qui sont considérés comme fiables par défaut ou par l'utilisateur. L'API est très similaire à `pickle`, et vous pouvez persister vos modèles comme expliqué dans la [documentation](https://skops.readthedocs.io/en/stable/persistence.html) en utilisant [`skops.io.dump`](https://skops.readthedocs.io/en/stable/modules/classes.html#skops.io.dump) et [`skops.io.dumps`](https://skops.readthedocs.io/en/stable/modules/classes.html#skops.io.dump).

In [ ]:
import skops.io as sio
obj = sio.dumps(clf)

Et vous pouvez les recharger en utilisant [`skops.io.load`](https://skops.readthedocs.io/en/stable/modules/classes.html#skops.io.load) et [`skops.io.loads`](https://skops.readthedocs.io/en/stable/modules/classes.html#skops.io.loads). Cependant, vous devez spécifier les types de données qui vous sont fiables. Vous pouvez obtenir les types inconnus existants dans un objet / fichier dumpé en utilisant [`skops.io.get_untrusted_types`](https://skops.readthedocs.io/en/stable/modules/classes.html#skops.io.get_untrusted_types), et après avoir vérifié son contenu, le passer à la fonction de chargement.

In [ ]:
unknown_types = sio.get_untrusted_types(obj)
clf = sio.loads(obj, trusted=unknown_types)

Si vous faites confiance à la source du fichier / objet, vous pouvez passer `trusted=True`.

In [ ]:
clf = sio.loads(obj, trusted=True)

Veuillez signaler les problèmes et les demandes de fonctionnalités liées à ce format sur le suivi des problèmes de skops.

<a id='interoperable-formats'></a>

# 9.2. Formats interopérables

Pour des besoins de reproductibilité et de contrôle de la qualité, lorsque différentes architectures et environnements doivent être pris en compte, l'exportation du modèle au format [Open Neural Network Exchange (ONNX)](https://onnx.ai/) ou au format [Predictive Model Markup Language (PMML)](https://dmg.org/pmml/v4-4-1/GeneralStructure.html) peut être une meilleure approche que l'utilisation de pickle seul. Celles-ci sont utiles lorsque vous souhaitez utiliser votre modèle pour la prédiction dans un environnement différent de celui dans lequel le modèle a été formé.

ONNX est une sérialisation binaire du modèle. Il a été développé pour améliorer l'utilisabilité de la représentation interopérable des modèles de données. Il vise à faciliter la conversion des modèles de données entre différents frameworks d'apprentissage automatique, et à améliorer leur portabilité sur différentes architectures informatiques. Plus de détails sont disponibles dans le [tutoriel ONNX](https://onnx.ai/get-started.html). Pour convertir le modèle scikit-learn en ONNX, un outil spécifique [sklearn-onnx](http://onnx.ai/sklearn-onnx/) a été développé.

PMML est une implémentation de la norme de document [XML](https://en.wikipedia.org/wiki/XML) définie pour représenter des modèles de données avec les données utilisées pour les générer. Étant lisible par l'homme et la machine, PMML est une bonne option pour la validation de modèles sur différentes plates-formes et l'archivage à long terme. D'autre part, comme XML en général, sa verbosité n'aide pas en production lorsque les performances sont critiques. Pour convertir le modèle scikit-learn en PMML, vous pouvez utiliser par exemple [sklearn2pmml](https://github.com/jpmml/sklearn2pmml) distribué sous la licence Affero GPLv3.